In [40]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:

%%writefile app1.py
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import pickle
import numpy as np
def load_model():
    with open('saved_steps.pkl', 'rb') as file:
        
        data = pickle.load(file)
        return data
data = load_model()

regressor = data["model"]
le_country = data["le_country"]
le_education = data["le_education"]
st.sidebar.header('please Maher inter input')
# def show_predict_page():
st.title('Software Developer Salary prediction')
#     st.header('saved_steps.pkl')
st.write("""### We need some information to predict salary""")

countries = (
    "United States",
    "India",
    "United Kingdom",
    "Germany",
    "Canada",
    "Brazil",
    "France",
    "Spain",
    "Australia",
    "Netherlands",
    "Poland",
    "Italy",
    "Russian Federation",
    "Sweden",
    )
education = (
    'Bachelor’s degree',
    'Master’s degree',
    'Less than a Bachelors',
    'Post grad',
    )
country = st.selectbox("Country", countries)
education = st.selectbox("Education Level", education)
experincence = st.slider("Years of Experience", 0, 50, 3)

ok = st.button("Calculate Salary")
if ok:
    X = np.array([[country, education, experincence ]])
    X[:, 0] = le_country.transform(X[:,0])
    X[:, 1] = le_education.transform(X[:,1])
    X = X.astype(float)

    salary =  regressor.predict(X)
    st.subheader(f"The estimated salary is ${salary[0]:2f}")

st.sidebar.selectbox("Explore or predict", ("Predict", "Explore"))

def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

def clean_experience(x):
    if x ==  'More than 50 years':
        return 50
    if x == 'Less than 1 year':
        return 0.5
    return float(x)

def clean_education(x):
    if 'Bachelor’s degree' in x:
        return 'Bachelor’s degree'
    if 'Master’s degree' in x:
        return 'Master’s degree'
    if 'Professional degree' in x or 'Other doctoral' in x:
        return 'Post grad'
    return 'Less than a Bachelors'
@st.cache

def load_data():
    df =pd.read_csv("survey_results_public.csv")
    
    df =df[["Country", "EdLevel", "YearsCodePro", "Employment", "ConvertedComp"]]

    
    df = df[df["ConvertedComp"].notnull()]
    df = df.dropna()
    
    df = df[df["Employment"] == "Employed full-time"]
    df = df.drop("Employment", axis=1)
    
    country_map = shorten_categories(df.Country.value_counts(), 400)
    df['Country'] = df['Country'].map(country_map)
    df.Country.value_counts()
    
    df = df[df["ConvertedComp"] <= 250000]
    df = df[df["ConvertedComp"] >= 10000]
    df = df[df['Country'] != 'Other']
    
    df["YearsCodePro"] = df["YearsCodePro"].apply(clean_experience)
    df["EdLevel"] = df["EdLevel"].apply(clean_education)
    df = df.rename({"ConvertedComp": "Salary"}, axis=1)
    return df

df = load_data()

st.title("Explor software Engnier")

st.write("""### stack overflow Developer Survey""")
st.write("""### Number of Data from deferent countries""")

data = df["Country"].value_counts()

fig1,ax1 = plt.subplots()
ax1.pie(data, labels=data.index, autopct = "%1.1f%%", shadow = True, startangle=90)
# ax1.axis("equale")
st.pyplot(fig1)


st.write("""### Mean Salary Based on Country""")

data = df.groupby(["Country"])["Salary"].mean().sort_values(ascending=True)

st.bar_chart(data)

st.write("""### Mean Salary Based on experience""")

data = df.groupby(["YearsCodePro"])["Salary"].mean().sort_values(ascending=True)

st.line_chart(data)
 





Overwriting app1.py


In [2]:
! streamlit run app1.py

^C
